##Configuration de l’environnement Spark

In [32]:
from pyspark.sql import SparkSession



In [33]:
spark = SparkSession.builder \
    .appName("Prediction") \
    .getOrCreate()


In [34]:
print(spark)


In [35]:
path = r"C:\Users\aamir\Desktop\YC\P\ChurnPrediction\src\infrastructure\data\dataset-68f599a4c9b84581895311-6907667a3f0be705109390.csv"

data = spark.read.csv(path, header=True, inferSchema=True)

data.printSchema()


root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [36]:
data.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93

In [40]:
num_columns =["RowNumber","CustomerId","CreditScore","Age","Tenure","Exited","EstimatedSalary","IsActiveMember","HasCrCard","NumOfProducts","Balance"];
obj_columns = ["Gender", "Geography", "Surname"]

data.select(obj_columns).describe().show()


+-------+------+---------+-------+
|summary|Gender|Geography|Surname|
+-------+------+---------+-------+
|  count| 10000|    10000|  10000|
|   mean|  NULL|     NULL|   NULL|
| stddev|  NULL|     NULL|   NULL|
|    min|Female|   France|  Abazu|
|    max|  Male|    Spain| Zuyeva|
+-------+------+---------+-------+



In [41]:
data.select(num_columns).describe().show()

+-------+------------------+-----------------+-----------------+------------------+------------------+-------------------+-----------------+-------------------+-------------------+------------------+-----------------+
|summary|         RowNumber|       CustomerId|      CreditScore|               Age|            Tenure|             Exited|  EstimatedSalary|     IsActiveMember|          HasCrCard|     NumOfProducts|          Balance|
+-------+------------------+-----------------+-----------------+------------------+------------------+-------------------+-----------------+-------------------+-------------------+------------------+-----------------+
|  count|             10000|            10000|            10000|             10000|             10000|              10000|            10000|              10000|              10000|             10000|            10000|
|   mean|            5000.5|  1.56909405694E7|         650.5288|           38.9218|            5.0128|             0.2037|100090

In [ ]:
from pyspark.sql.functions import col, count, when, trim, isnan
from pyspark.sql.types import FloatType, DoubleType, DecimalType

null_counts = []
for c in data.columns:
    dtype = next(f.dataType for f in data.schema.fields if f.name == c)

    if isinstance(dtype, (FloatType, DoubleType, DecimalType)):
        cond = col(c).isNull() | isnan(col(c))
    else:
        cond = col(c).isNull() | (trim(col(c)) == "")
        
    null_counts.append(count(when(cond, c)).alias(c))

nulldata = data.select(null_counts)
nulldata.show()

+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|        0|         0|      0|          0|        0|     0|  0|     0|      0|            0|        0|             0|              0|     0|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+



In [51]:
for c in num_columns:
    qa = data.approxQuantile(c, [0.25, 0.75], 0.01)  
    q1, q3 = qa[0], qa[1]
    iqr = q3 - q1
    print(f"Q1: {q1}, Q3: {q3}, IQR: {iqr}")


Q1: 2496.0, Q3: 7408.0, IQR: 4912.0
Q1: 15628303.0, Q3: 15750929.0, IQR: 122626.0
Q1: 584.0, Q3: 714.0, IQR: 130.0
Q1: 32.0, Q3: 44.0, IQR: 12.0
Q1: 2.0, Q3: 7.0, IQR: 5.0
Q1: 0.0, Q3: 0.0, IQR: 0.0
Q1: 50906.65, Q3: 147342.03, IQR: 96435.38
Q1: 0.0, Q3: 1.0, IQR: 1.0
Q1: 0.0, Q3: 1.0, IQR: 1.0
Q1: 1.0, Q3: 2.0, IQR: 1.0
Q1: 0.0, Q3: 126615.94, IQR: 126615.94
